In [1]:
import time
startTime = time.time()

import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io
import math
import glob
import matplotlib.image as mpimg
import os
from exif import Image

directory = 'C:/Users/abordas/Desktop/GrainSegmentation/HED/'

In [11]:
for folder in os.walk(directory):
    for filename in folder[-1]:
        if filename[-4:] == '.jpg':  
            img = Image(directory + filename[:-4]+ '/'+filename)  
            if img.has_exif == True:
                print(filename + ' has metadata')
                pixels_to_um = 1/float(img.get('user_comment'))
                print('Pixel ratio is: ' + str(img.get('user_comment') + ' pixels per um'))
            else:
                print(filename + ' does not contain metadata')
                print("_______________________________________________________________________________________________________")
                continue     #meant for large processing: lack of metadata does not stop batches
        if filename[-4:] == '.png':

            img = cv2.imread(directory+filename[:-15]+ '/'+filename)

            plt.rcParams['figure.figsize'] = [20, 10]
                        
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = ~gray
    
    
            #Threshold
            ret, thresh = cv2.threshold(gray,250, 255,cv2.THRESH_BINARY)
    
            from skimage.segmentation import clear_border
            thresh = clear_border(thresh) #Remove edge touching
    
    
            from skimage.morphology import disk
            #kernel = disk(1)
            kernel = np.ones((3,3),np.uint8)

            ite = 5  

            #method = 'gray_dilate_only'
            #method = 'gray_erode+dilate'
            method = 'thresh'
            #method = ''       #only thresholding
            #dilated = thresh      #only thresholding

            if method == 'gray_dilate_only':
                dilated = cv2.dilate(gray,kernel,iterations = ite)
            elif method == 'gray_erode+dilate':
                eroded = cv2.erode(gray,kernel,iterations = ite)
                dilated = cv2.dilate(eroded,kernel,iterations = ite)
            elif method == 'thresh': 
                eroded = cv2.erode(thresh,kernel,iterations = ite+9)
                dilated = cv2.dilate(eroded,kernel,iterations = ite+8)
        
        
            mask = dilated == 255
    
    
            s = [[1,1,1],[1,1,1],[1,1,1]] # This is what ImageJ uses for structural objects
            labeled_mask, num_labels = ndimage.label(mask, structure=s)
    

            img2 = color.label2rgb(labeled_mask, bg_label=0)
    
    
            clusters = measure.regionprops(labeled_mask, img) 
    
    
            area = np.zeros(len(clusters))
            equivalent_diameter = np.zeros(len(clusters))
            for idx, cluster_props in enumerate(clusters):
                area[idx] = cluster_props['Area'] * pixels_to_um**2
                equivalent_diameter[idx] = cluster_props['equivalent_diameter']* pixels_to_um
        

            np.set_printoptions(suppress=True)
            #print(area)
            print("Average grain area is {}; std is {}".format(np.mean(area), np.std(area)))
    
    
            circle_approx = np.mean(equivalent_diameter)
            hex_out = 2*math.sqrt((2*np.mean(area))/(3*math.sqrt(3)))
            a = math.sqrt(2*np.mean(area)/(3*math.sqrt(3)))
            hex_in = 2*math.sqrt(a*a-(a/2)*(a/2))
            hex_avg = np.mean([hex_in, hex_out])
            print('Average grain size using circle approximation, in micrometers, is', circle_approx, '.')
            print('Average grain size using hexagonal approximation, in micrometers, is', hex_avg, '.')
            print('Standard deviation of individual areas is', np.std(area), '.')
            print('Number of cells:', len(clusters))
    
    
            #plt.hist(equivalent_diameter.flat, bins=30)
            #plt.xlabel('Grain Size um')
            #plt.ylabel('Frequency')
            #plt.title('Circle Approx Grain Size Distribution')

            #plt.hist(area.flat, bins=30)
            #plt.xlabel('Area (pixels)')
            #plt.ylabel('Frequency')
            #plt.title('Pixel Count Per Grain')
    
            print("_______________________________________________________________________________________________________")
    
    
executionTime = (time.time() - startTime)
print(executionTime)


10x_CeO2_2hrs_1500C_6_sint_10_cropped_metadata.jpg has metadata
Pixel ratio is: 1.8233 pixels per um
Average grain area is 3607.685168431191; std is 4741.575773450707
Average grain size using circle approximation, in micrometers, is 60.15627137296503 .
Average grain size using hexagonal approximation, in micrometers, is 69.53539469907986 .
Standard deviation of individual areas is 4741.575773450707 .
Number of cells: 104
_______________________________________________________________________________________________________
10x_CeO2_2hrs_1500C_6_sint_11_cropped_metadata.jpg has metadata
Pixel ratio is: 1.8233 pixels per um
Average grain area is 2631.452684547464; std is 2712.295751546894
Average grain size using circle approximation, in micrometers, is 52.60401092579272 .
Average grain size using hexagonal approximation, in micrometers, is 59.38672712073036 .
Standard deviation of individual areas is 2712.295751546894 .
Number of cells: 130
______________________________________________